# <b>Python for Data Analysis</b>
# 10.  Data Aggregation and Group Operations

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels as sm
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.set_printoptions(precision = 4, suppress = True)
from pandas import Series, DataFrame
%matplotlib inline

## 10.1 How to Think About Group Operations

In [2]:
df = pd.DataFrame({"key1":  ["a", "a", None, "b", "b", "a", None],
                   "key2":  pd.Series([1, 2, 1, 2, 1, None, 1], dtype = "Int64"),
                   "data1": np.random.standard_normal(7),
                   "data2": np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,-1.019386,0.016573
1,a,2,-0.285176,0.387607
2,None,1,0.503882,1.560638
3,b,2,0.247794,0.063522
4,b,1,0.036166,1.188258
5,a,<NA>,1.343940,-0.522904
6,None,1,0.551140,0.601258


In [3]:
grouped = df["data1"].groupby(df["key1"])
grouped

In [4]:
grouped.mean()

key1
a    0.013126
b    0.141980
Name: data1, dtype: float64

In [5]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     1      -1.019386
      2      -0.285176
b     1       0.036166
      2       0.247794
Name: data1, dtype: float64

In [6]:
means.unstack()

key2,1,2
key1,,
a,-1.019386,-0.285176
b,0.036166,0.247794


In [7]:
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]

In [8]:
states

array(['OH', 'CA', 'CA', 'OH', 'OH', 'CA', 'OH'], dtype='<U2')

In [9]:
years

[2005, 2005, 2006, 2005, 2006, 2005, 2006]

In [10]:
df["data1"].groupby([states, years]).mean()

CA  2005    0.529382
    2006    0.503882
OH  2005   -0.385796
    2006    0.293653
Name: data1, dtype: float64

In [11]:
df

,key1,key2,data1,data2
0,a,1,-1.019386,0.016573
1,a,2,-0.285176,0.387607
2,None,1,0.503882,1.560638
3,b,2,0.247794,0.063522
4,b,1,0.036166,1.188258
5,a,<NA>,1.343940,-0.522904
6,None,1,0.551140,0.601258


In [12]:
df.groupby("key1").mean()

,key2,data1,data2
key1,,,
a,1.5,0.013126,-0.039575
b,1.5,0.141980,0.625890


In [13]:
df.groupby("key2").mean(numeric_only = True)

,data1,data2
key2,,
1,0.017951,0.841682
2,-0.018691,0.225565


In [14]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1    -1.019386  0.016573
     2    -0.285176  0.387607
b    1     0.036166  1.188258
     2     0.247794  0.063522

In [15]:
df.groupby(["key1", "key2"], dropna = False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [16]:
df.groupby("key1", dropna = False).size()

key1
a      3
b      2
NaN    2
dtype: int64

In [17]:
df.groupby(["key1", "key2"], dropna = False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [18]:
df.groupby(["key1", "key2"], dropna = False).count()

data1  data2
key1 key2              
a    1         1      1
     2         1      1
     <NA>      1      1
b    1         1      1
     2         1      1
NaN  1         2      2

In [19]:
df.groupby(["key1", "key2"], dropna = False).count()

data1  data2
key1 key2              
a    1         1      1
     2         1      1
     <NA>      1      1
b    1         1      1
     2         1      1
NaN  1         2      2

In [20]:
df.groupby(["key1", "key2"], dropna = False).count()

data1  data2
key1 key2              
a    1         1      1
     2         1      1
     <NA>      1      1
b    1         1      1
     2         1      1
NaN  1         2      2

### Iterating over Groups

In [21]:
for name, group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1 -1.019386  0.016573
1    a     2 -0.285176  0.387607
5    a  <NA>  1.343940 -0.522904
b
  key1  key2     data1     data2
3    b     2  0.247794  0.063522
4    b     1  0.036166  1.188258


In [22]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)

('a', 1)
  key1  key2     data1     data2
0    a     1 -1.019386  0.016573
('a', 2)
  key1  key2     data1     data2
1    a     2 -0.285176  0.387607
('b', 1)
  key1  key2     data1     data2
4    b     1  0.036166  1.188258
('b', 2)
  key1  key2     data1     data2
3    b     2  0.247794  0.063522


In [23]:
pieces = {name: group for name, group in df.groupby("key1")}
pieces["b"]

,key1,key2,data1,data2
3,b,2,0.247794,0.063522
4,b,1,0.036166,1.188258


In [24]:
grouped = df.groupby({"key1": "key", "key2": "key",
                      "data1": "data", "data2": "data"},
                     axis = "columns")

In [25]:
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)

data
      data1     data2
0 -1.019386  0.016573
1 -0.285176  0.387607
2  0.503882  1.560638
3  0.247794  0.063522
4  0.036166  1.188258
5  1.343940 -0.522904
6  0.551140  0.601258
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


### Selecting a Column or Subset of Columns

In [26]:
df.groupby("key1")["data1"]
df.groupby("key1")[["data2"]]

In [27]:
df["data1"].groupby(df["key1"])
df[["data2"]].groupby(df["key1"])

In [28]:
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1     0.016573
     2     0.387607
b    1     1.188258
     2     0.063522

In [29]:
s_grouped = df.groupby(["key1", "key2"])["data2"]
s_grouped

In [30]:
s_grouped.mean()

key1  key2
a     1       0.016573
      2       0.387607
b     1       1.188258
      2       0.063522
Name: data2, dtype: float64

### Grouping with Dictionaries and Series

In [31]:
people = pd.DataFrame(np.random.standard_normal((5, 5)),
                      columns = ["a", "b", "c", "d", "e"],
                      index = ["Joe", "Steve", "Wanda", "Jill", "Trey"])
people

,a,b,c,d,e
Joe,0.116809,2.437733,1.260865,0.721246,-0.649159
Steve,2.310995,-1.179889,1.097618,1.029850,0.566734
Wanda,-0.563118,-0.635871,-0.621563,-1.294428,0.120058
Jill,-0.273222,0.440409,0.032114,0.014747,-0.702870
Trey,0.373005,0.200002,0.316077,-1.011452,-0.337659


In [32]:
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,0.116809,2.437733,1.260865,0.721246,-0.649159
Steve,2.310995,-1.179889,1.097618,1.029850,0.566734
Wanda,-0.563118,NaN,NaN,-1.294428,0.120058
Jill,-0.273222,0.440409,0.032114,0.014747,-0.702870
Trey,0.373005,0.200002,0.316077,-1.011452,-0.337659


In [33]:
mapping = {"a": "red", "b": "red", "c": "blue",
           "d": "blue", "e": "red", "f" : "orange"}
mapping

{'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

In [34]:
by_column = people.groupby(mapping, axis = "columns")
by_column.sum()

,blue,red
Joe,1.982111,1.905383
Steve,2.127469,1.697840
Wanda,-1.294428,-0.443060
Jill,0.046861,-0.535683
Trey,-0.695375,0.235348


In [35]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [36]:
people.groupby(map_series, axis = 1).count()

,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


### Grouping with Functions

In [37]:
people

,a,b,c,d,e
Joe,0.116809,2.437733,1.260865,0.721246,-0.649159
Steve,2.310995,-1.179889,1.097618,1.029850,0.566734
Wanda,-0.563118,NaN,NaN,-1.294428,0.120058
Jill,-0.273222,0.440409,0.032114,0.014747,-0.702870
Trey,0.373005,0.200002,0.316077,-1.011452,-0.337659


In [38]:
people.groupby(len).sum()

,a,b,c,d,e
3,0.116809,2.437733,1.260865,0.721246,-0.649159
4,0.099783,0.640411,0.348191,-0.996705,-1.040528
5,1.747877,-1.179889,1.097618,-0.264577,0.686792


In [39]:
key_list = ["one", "one", "one", "two", "two"]

In [40]:
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,0.116809,2.437733,1.260865,0.721246,-0.649159
4,two,-0.273222,0.200002,0.032114,-1.011452,-0.702870
5,one,-0.563118,-1.179889,1.097618,-1.294428,0.120058


### Grouping by Index Levels

In [41]:
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"],
                                     [1, 3, 5, 1, 3]],
                                    names = ["cty", "tenor"])
columns

MultiIndex([('US', 1),
            ('US', 3),
            ('US', 5),
            ('JP', 1),
            ('JP', 3)],
           names=['cty', 'tenor'])

In [42]:
hier_df = pd.DataFrame(np.random.standard_normal((4, 5)), 
                       columns = columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.252324  0.122485  0.177380 -0.773006 -0.636109
1     -1.100391 -0.903351  1.331259  1.471134 -1.138657
2     -1.038577 -1.117088 -1.035116 -0.813157  0.784926
3      3.192570 -1.316659  0.191466 -0.269919 -0.317499

In [43]:
hier_df.groupby(level = "cty", axis = 1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 Data Aggregation

In [44]:
df

,key1,key2,data1,data2
0,a,1,-1.019386,0.016573
1,a,2,-0.285176,0.387607
2,None,1,0.503882,1.560638
3,b,2,0.247794,0.063522
4,b,1,0.036166,1.188258
5,a,<NA>,1.343940,-0.522904
6,None,1,0.551140,0.601258


In [45]:
grouped = df.groupby("key1")
grouped

In [46]:
grouped["data1"].nsmallest(2)

key1   
a     0   -1.019386
      1   -0.285176
b     4    0.036166
      3    0.247794
Name: data1, dtype: float64

In [47]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [48]:
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,2.363327,0.910511
b,1,0.211628,1.124735


In [49]:
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.013126  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0  0.141980  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     0.529382  1.343940   3.0 -0.039575  0.457845 -0.522904 -0.253165   
b     0.194887  0.247794   2.0  0.625890  0.795308  0.063522  0.344706   

                                    
           50%       75%       max  
key1                                
a     0.016573  0.202090  0.387607  
b     0.625890  0.907074  1.188258  

[2 rows x 24 columns]

### Column-Wise and Multiple Function Application

In [50]:
tips = pd.read_csv("examples/tips.csv")

In [51]:
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [52]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [53]:
grouped = tips.groupby(["day", "smoker"])

In [54]:
grouped_pct = grouped["tip_pct"]

In [55]:
grouped_pct.agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [56]:
tips.groupby(["day", "smoker"])["tip_pct"].agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [57]:
tips.groupby(["day", "smoker"])["tip_pct"].mean()

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [58]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [59]:
grouped_pct.agg([("average", "mean"), ("stdev", np.std)]) # (column name, function)

average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [60]:
grouped_pct.agg([("average", "mean"), ("stdev", "std")]) # (column name, function)

average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [61]:
grouped_pct.agg(["mean", "std", ("Max/min diff.", peak_to_peak)])

mean       std  Max/min diff.
day  smoker                                   
Fri  No      0.151650  0.028123       0.067349
     Yes     0.174783  0.051293       0.159925
Sat  No      0.158048  0.039767       0.235193
     Yes     0.147906  0.061375       0.290095
Sun  No      0.160113  0.042347       0.193226
     Yes     0.187250  0.154134       0.644685
Thur No      0.160298  0.038774       0.193350
     Yes     0.163863  0.039389       0.151240

In [62]:
functions = ["count", "mean", "max"]

In [63]:
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [64]:
result["tip_pct"]

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [65]:
ftuples = [("Average", "mean"), ("Variance", np.var)]

In [66]:
grouped[["tip_pct", "total_bill"]].agg(ftuples)

tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [67]:
grouped.agg({"tip": np.max, "size": "sum"})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [68]:
grouped.agg({"tip_pct": ["min", "max", "mean", "std"],
             "size": "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data Without Row Indexes

In [69]:
grouped = tips.groupby(["day", "smoker"], as_index = False)
grouped.mean(numeric_only = True)

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [70]:
result_test = grouped.agg({"tip_pct": ["min", "max", "mean", "std"],
                           "size": "sum"})
result_test.reset_index()

index   day smoker   tip_pct                               size
                           min       max      mean       std  sum
0     0   Fri     No  0.120385  0.187735  0.151650  0.028123    9
1     1   Fri    Yes  0.103555  0.263480  0.174783  0.051293   31
2     2   Sat     No  0.056797  0.291990  0.158048  0.039767  115
3     3   Sat    Yes  0.035638  0.325733  0.147906  0.061375  104
4     4   Sun     No  0.059447  0.252672  0.160113  0.042347  167
5     5   Sun    Yes  0.065660  0.710345  0.187250  0.154134   49
6     6  Thur     No  0.072961  0.266312  0.160298  0.038774  112
7     7  Thur    Yes  0.090014  0.241255  0.163863  0.039389   40

## 10.3 Apply: General split-apply-combine

In [71]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [72]:
def top(df, n = 5, column = "tip_pct"):
    return df.sort_values(column, ascending = False)[:n]

In [73]:
top(tips, n = 6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [74]:
tips.sort_values("tip", ascending = False)[:5]

,total_bill,tip,smoker,day,time,size,tip_pct
170,50.81,10.00,Yes,Sat,Dinner,3,0.196812
212,48.33,9.00,No,Sat,Dinner,4,0.186220
23,39.42,7.58,No,Sat,Dinner,4,0.192288
59,48.27,6.73,No,Sat,Dinner,4,0.139424
141,34.30,6.70,No,Thur,Lunch,6,0.195335


In [75]:
tips.groupby("smoker").apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [76]:
tips.groupby(["smoker", "day"]).apply(top, n = 1, column = "total_bill")

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [77]:
result = tips.groupby("smoker")["tip_pct"].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [78]:
result.unstack("smoker")

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

This code is alternative to <code>.describe()</code>:

In [79]:
# def f(group):
#    return group.describe()

# grouped.apply(f)

### Suppressing the Group Keys

In [80]:
tips.groupby("smoker", group_keys = False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
232,11.61,3.39,No,Sat,Dinner,2,0.291990
149,7.51,2.00,No,Thur,Lunch,2,0.266312
51,10.29,2.60,No,Sun,Dinner,2,0.252672
185,20.69,5.00,No,Sun,Dinner,5,0.241663
88,24.71,5.85,No,Thur,Lunch,2,0.236746
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [81]:
tips.groupby("smoker").apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

### Quantile and Bucket Analysis

In [82]:
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                      "data2": np.random.standard_normal(1000)})
frame.head()

,data1,data2
0,-0.572786,0.360203
1,0.576441,0.094488
2,-0.413744,0.556943
3,0.025763,-1.150811
4,1.593066,0.423401


In [83]:
quartiles = pd.cut(frame["data1"], 4)
quartiles.head(10)

0    (-1.83, -0.205]
1    (-0.205, 1.421]
2    (-1.83, -0.205]
3    (-0.205, 1.421]
4     (1.421, 3.046]
5    (-0.205, 1.421]
6    (-3.462, -1.83]
7    (-1.83, -0.205]
8    (-0.205, 1.421]
9    (-1.83, -0.205]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.462, -1.83] < (-1.83, -0.205] < (-0.205, 1.421] < (1.421, 3.046]]

In [84]:
def get_stats(group):
    return pd.DataFrame(
        {"min": group.min(), "max": group.max(),
         "count": group.count(), "mean": group.mean()})

In [85]:
grouped = frame.groupby(quartiles)

In [86]:
grouped.apply(get_stats)

min       max  count      mean
data1                                                     
(-3.462, -1.83] data1 -3.455548 -1.831707     33 -2.143444
                data2 -1.738424  2.029942     33  0.209843
(-1.83, -0.205] data1 -1.826930 -0.207344    411 -0.830952
                data2 -2.450568  3.450273    411  0.040653
(-0.205, 1.421] data1 -0.199487  1.394964    477  0.499578
                data2 -3.935918  3.093598    477  0.015813
(1.421, 3.046]  data1  1.425454  3.046066     79  1.846997
                data2 -3.194043  2.022896     79  0.008480

In [88]:
grouped.agg(["min", "max", "count", "mean"])

data1                               data2                  \
                      min       max count      mean       min       max count   
data1                                                                           
(-3.462, -1.83] -3.455548 -1.831707    33 -2.143444 -1.738424  2.029942    33   
(-1.83, -0.205] -1.826930 -0.207344   411 -0.830952 -2.450568  3.450273   411   
(-0.205, 1.421] -0.199487  1.394964   477  0.499578 -3.935918  3.093598   477   
(1.421, 3.046]   1.425454  3.046066    79  1.846997 -3.194043  2.022896    79   

                           
                     mean  
data1                      
(-3.462, -1.83]  0.209843  
(-1.83, -0.205]  0.040653  
(-0.205, 1.421]  0.015813  
(1.421, 3.046]   0.008480

In [89]:
quartiles_samp = pd.qcut(frame["data1"], 4)
quartiles_samp.head()

0    (-0.752, -0.0454]
1     (-0.0454, 0.671]
2    (-0.752, -0.0454]
3     (-0.0454, 0.671]
4       (0.671, 3.046]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.457, -0.752] < (-0.752, -0.0454] < (-0.0454, 0.671] < (0.671, 3.046]]

In [90]:
grouped = frame.groupby(quartiles_samp)
grouped.apply(get_stats)

min       max  count      mean
data1                                                       
(-3.457, -0.752]  data1 -3.455548 -0.752740    250 -1.282673
                  data2 -2.450568  2.733143    250  0.051945
(-0.752, -0.0454] data1 -0.751300 -0.045377    250 -0.394570
                  data2 -2.828355  3.450273    250  0.059402
(-0.0454, 0.671]  data1 -0.045363  0.670284    250  0.299390
                  data2 -3.935918  3.093598    250 -0.007731
(0.671, 3.046]    data1  0.672490  3.046066    250  1.265679
                  data2 -3.200225  2.096245    250  0.023770

In [91]:
quartiles_samp = pd.qcut(frame["data1"], 4)
quartiles_samp.head()

0    (-0.752, -0.0454]
1     (-0.0454, 0.671]
2    (-0.752, -0.0454]
3     (-0.0454, 0.671]
4       (0.671, 3.046]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.457, -0.752] < (-0.752, -0.0454] < (-0.0454, 0.671] < (0.671, 3.046]]

### Example: Filling Missing Values with Group-Specific Values

In [92]:
s = pd.Series(np.random.standard_normal(6))
s

0   -0.548530
1    0.587203
2    1.136285
3    1.777758
4   -0.846201
5   -0.884020
dtype: float64

In [93]:
s[::2] = np.nan
s

0         NaN
1    0.587203
2         NaN
3    1.777758
4         NaN
5   -0.884020
dtype: float64

In [94]:
s.fillna(s.mean())

0    0.493647
1    0.587203
2    0.493647
3    1.777758
4    0.493647
5   -0.884020
dtype: float64

In [95]:
states = ["Ohio", "New York", "Vermont", "Florida",
          "Oregon", "Nevada", "California", "Idaho"]

group_key = ["East", "East", "East", "East",
             "West", "West", "West", "West"]

In [96]:
data = pd.Series(np.random.standard_normal(8), index = states)
data

Ohio         -0.246381
New York      0.967467
Vermont       1.424880
Florida      -0.680887
Oregon       -1.529108
Nevada       -0.277682
California   -0.043155
Idaho        -0.995226
dtype: float64

In [97]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

Ohio         -0.246381
New York      0.967467
Vermont            NaN
Florida      -0.680887
Oregon       -1.529108
Nevada             NaN
California   -0.043155
Idaho              NaN
dtype: float64

In [98]:
data.groupby(group_key).mean()

East    0.013400
West   -0.786132
dtype: float64

In [99]:
data.groupby(group_key).mean()

East    0.013400
West   -0.786132
dtype: float64

In [100]:
data.groupby(group_key).mean()

East    0.013400
West   -0.786132
dtype: float64

In [101]:
def fill_mean(group):
    return group.fillna(group.mean())

In [102]:
data.groupby(group_key).apply(fill_mean)

Ohio         -0.246381
New York      0.967467
Vermont       0.013400
Florida      -0.680887
Oregon       -1.529108
Nevada       -0.786132
California   -0.043155
Idaho        -0.786132
dtype: float64

In [103]:
fill_values = {"East": 0.5, "West": -1}

In [104]:
def fill_func(group):
    return group.fillna(fill_values[group.name])

In [105]:
data.groupby(group_key).apply(fill_func)

Ohio         -0.246381
New York      0.967467
Vermont       0.500000
Florida      -0.680887
Oregon       -1.529108
Nevada       -1.000000
California   -0.043155
Idaho        -1.000000
dtype: float64

### Example: Random Sampling and Permutation

In [106]:
suits = list("HSCD")
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + list("JKQ")
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)
    
deck = pd.Series(card_val, index = cards)

In [107]:
deck

AH      1
2H      2
3H      3
4H      4
5H      5
       ..
9D      9
10D    10
JD     10
KD     10
QD     10
Length: 52, dtype: int64

In [108]:
deck.head(13)

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [109]:
def draw(deck, n = 5):
    return deck.sample(n) # return a n-number of values from a df

In [110]:
draw(deck)

5C     5
7C     7
9H     9
6C     6
QC    10
dtype: int64

In [111]:
deck.sample(10)

QH    10
JD    10
QS    10
6H     6
5H     5
7C     7
5C     5
4H     4
7D     7
KC    10
dtype: int64

In [112]:
def get_suit(card):
    # last letter is suit
    return card[-1]

In [113]:
deck.groupby(get_suit).apply(draw, n = 2)

C  AC     1
   6C     6
D  7D     7
   8D     8
H  6H     6
   KH    10
S  9S     9
   6S     6
dtype: int64

In [114]:
deck.groupby(get_suit, group_keys = False).apply(draw, n = 2)

3C     3
4C     4
9D     9
4D     4
AH     1
8H     8
9S     9
JS    10
dtype: int64

### Example: Group Weighted Average and Correlation

In [115]:
df = pd.DataFrame({"category": ["a", "a", "a", "a",
                                "b", "b", "b", "b"],
                   "data": np.random.standard_normal(8),
                   "weights": np.random.uniform(size = 8)})
df

,category,data,weights
0,a,0.802104,0.945572
1,a,1.443559,0.085064
2,a,-0.375251,0.697911
3,a,-1.521867,0.061551
4,b,-0.380000,0.797416
5,b,0.046685,0.336522
6,b,-0.930164,0.575125
7,b,-0.311707,0.422003


In [116]:
grouped = df.groupby("category")

In [117]:
def get_wavg(group):
    return np.average(group["data"], weights = group["weights"])

In [118]:
grouped.apply(get_wavg)

category
a    0.293658
b   -0.447574
dtype: float64

In [119]:
get_wavg(df[df.category == "b"])

-0.4475740494227778

In [120]:
get_wavg(df[df.category == "b"])

-0.4475740494227778

In [121]:
get_wavg(df[df.category == "b"])

-0.4475740494227778

In [122]:
close_px = pd.read_csv("examples/stock_px.csv",
                       parse_dates = True,
                       index_col = 0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [123]:
close_px.tail(4)

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [124]:
close_px.tail(4)

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [125]:
def spx_corr(group):
    return group.corrwith(group["SPX"])

In [129]:
close_px.head()

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93


In [130]:
rets = close_px.pct_change().dropna()
rets.head()

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386


In [131]:
def get_year(x):
    return x.year

In [132]:
by_year = rets.groupby(get_year)

In [133]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [136]:
def corr_aapl_msft(group):
    return group["AAPL"].corr(group["MSFT"])

In [137]:
by_year.apply(corr_aapl_msft)

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Example: Group-Wise Linear Regression

In [139]:
import statsmodels.api as sm

def regress(data, yvar = None, xvars = None):
    Y = data[yvar]
    X = data[xvars]
    X["intercept"] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [140]:
by_year.apply(regress, yvar = "AAPL", xvars = ["SPX"])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 10.4 Group Transforms and "Unwrapped" GroupBys

In [141]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [144]:
g = df.groupby("key")["value"]
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [146]:
def get_mean(group):
    return group.mean()

In [147]:
g.transform(get_mean)

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [149]:
g.transform("mean")

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [150]:
def times_two(group):
    return group * 2

In [151]:
g.transform(times_two)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [152]:
def get_ranks(group):
    return group.rank(ascending = False)

In [153]:
g.transform(get_ranks)

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [154]:
def normalize(x):
    return (x - x.mean()) / x.std()

In [155]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [156]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [158]:
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [157]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

## 10.5 Pivot Tables and Cross-Tabulation